In [1]:
# Importing neccessary libraries
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.animation import FuncAnimation

In [2]:
def IsothermalShockTube(nt, cfl_factor=0.5):
    %matplotlib tk
    
    # Define parameters
    a, b = -10, 10
    nx = 100
    x = np.linspace(a, b, nx)
    dx = x[1] - x[0]
    t = 0
    dt = cfl_factor * dx / 1.0  # Use isothermal sound speed cs = 1
    
    # Initial conditions
    theta_function = np.piecewise(x, [x <= 0, x > 0], [0, 1])
    rho0 = 2 * theta_function + 1
    v0 = np.zeros_like(x)

    # Plotting initial condition
    fig, ax = plt.subplots()
    line_rho = ax.plot(x, rho0, label="Density")[0]
    ax.set_xlabel(r"$x$")
    ax.set_ylabel(r"Value")
    ax.set_title("Isothermal Shock Tube")

    # Set y-axis limits
    ax.set_ylim(-3, 6)

    # Copying the initial conditions for the array size and later updating them based on the function
    rho = rho0.copy()
    v = v0.copy()

    # Define flux limiter function
    def flux_limiter(r):
        return max(0, min(1, r))

    # Define Algorithm function
    def Algorithm(frame):
        nonlocal t, rho, v

        # Calculate new timestep based on CFL condition
        dt = cfl_factor * dx / max(np.max(np.abs(v)), 1.0)

        # Transport step (momentum equation)
        for i in range(nx):
            dm = rho[i % nx] - rho[(i-1) % nx]
            dp = rho[(i+1) % nx] - rho[i % nx]
            rm = 0 if dm == 0 else (rho[(i-1) % nx] - rho[(i-2) % nx]) / dm
            rp = 0 if dp == 0 else (rho[i % nx] - rho[(i-1) % nx]) / dp

            phi_plus = flux_limiter(rp)
            phi_minus = flux_limiter(rm)

            fp = rho[i] * v[i] + abs(v[i]) * (1 - abs(v[i] * dt / dx)) * phi_plus
            fm = rho[(i-1) % nx] * v[(i-1) % nx] + abs(v[(i-1) % nx]) * (1 - abs(v[(i-1) % nx] * dt / dx)) * phi_minus

            rho[i] += (dt / dx) * (fm - fp)

        # Source step (pressure force equation)
        for i in range(nx):
            rho[i] += -dt * (rho[i] * v[i])

        # Velocity update
        for i in range(nx):
            v[i] = rho[i] / max(1e-6, rho[i])  # Avoid division by zero

        # Update the plot with the new solution
        line_rho.set_ydata(rho)

        t += dt
        return line_rho

    # Create animation
    anim = FuncAnimation(fig, Algorithm, frames=nt, interval=50)
    
    # Return animation
    return anim

In [3]:
# Run the simulation
IsothermalShockTube(nt=600, cfl_factor=0.5)